In [1]:
import os, pathlib, time, copy, librosa, gc

import torch
import torchaudio
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler

from rich_logger import RichTablePrinter
from tqdm import trange
from rich import pretty
from rich.console import Console
from IPython.display import Audio

console = Console()
pretty.install()
plt.ion()

use_gpu = torch.cuda.is_available()
if use_gpu:
    device = 'cuda'
    print("Using CUDA")
else:
    device = 'cpu'
    print('Using CPU')

Using CUDA


In [2]:
class GreedyCTCDecoder(torch.nn.Module):
    def __init__(self, labels, blank=0):
        super().__init__()
        self.labels = labels
        self.blank = blank

    def forward(self, emission: torch.Tensor) -> str:
        """Given a sequence emission over labels, get the best path string
        Args:
          emission (Tensor): Logit tensors. Shape `[num_seq, num_label]`.

        Returns:
          str: The resulting transcript
        """
        indices = torch.argmax(emission, dim=-1)  # [num_seq,]
        indices = torch.unique_consecutive(indices, dim=-1)
        indices = [i for i in indices if i != self.blank]
        return "".join([self.labels[i] for i in indices])

In [3]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model().to(device)

In [4]:
test_file = './datasets/speech-handsign_commands/speech/val/down/bf5d409d_nohash_0.wav'
wav, sr = torchaudio.load(test_file)
Audio(wav.numpy()[0], rate=sr)
wav = wav.to(device)

In [59]:
down_1 = 'datasets/speech-handsign_commands/speech/val/down/c0445658_nohash_3.wav'
down_2 = 'datasets/speech-handsign_commands/speech/val/down/c4e1f6e0_nohash_1.wav'
right_1 = 'datasets/speech-handsign_commands/speech/val/right/c44d2a58_nohash_2.wav'
right_2 = 'datasets/speech-handsign_commands/speech/val/right/c6a23ff5_nohash_2.wav'
stop_1 = 'datasets/speech-handsign_commands/speech/val/stop/bf8d5617_nohash_0.wav'
stop_2 = 'datasets/speech-handsign_commands/speech/val/stop/cc6bae0d_nohash_0.wav'


def pad_seq(seq):
    num_frames = seq.shape[1]
    if num_frames > 16000:
        seq = seq[:, :16000]
    else:
        pad_width = 16000 - num_frames
        seq = np.pad(seq, [(0, 0), (0, pad_width)], mode="constant")
    return seq


wav_down_1 = pad_seq(torchaudio.load(down_1)[0])
wav_down_2 = pad_seq(torchaudio.load(down_2)[0])
wav_right_1 = pad_seq(torchaudio.load(right_1)[0])
wav_right_2 = pad_seq(torchaudio.load(right_2)[0])
wav_stop_1 = pad_seq(torchaudio.load(stop_1)[0])
wav_stop_2 = pad_seq(torchaudio.load(stop_2)[0])

wav_down_1 = torch.from_numpy(wav_down_1).to(device)
wav_down_2 = torch.from_numpy(wav_down_2).to(device)
wav_right_1 = torch.from_numpy(wav_right_1).to(device)
wav_right_2 = torch.from_numpy(wav_right_2).to(device)
wav_stop_1 = torch.from_numpy(wav_stop_1).to(device)
wav_stop_2 = torch.from_numpy(wav_stop_2).to(device)

with torch.inference_mode():
    down_features_1, _ = model(wav_down_1)
    down_features_2, _ = model(wav_down_2)
    right_features_1, _ = model(wav_right_1)
    right_features_2, _ = model(wav_right_2)
    stop_features_1, _ = model(wav_stop_1)
    stop_features_2, _ = model(wav_stop_2)
down_features_1 = down_features_1[0].squeeze(0).flatten()#.mean(dim=0)
down_features_2 = down_features_2[0].squeeze(0).flatten()#.mean(dim=0)
right_features_1 = right_features_1[0].squeeze(0).flatten()#.mean(dim=0)
right_features_2 = right_features_1[0].squeeze(0).flatten()#.mean(dim=0)
stop_features_1 = stop_features_1[0].squeeze(0).flatten()#.mean(dim=0)
stop_features_2 = stop_features_2[0].squeeze(0).flatten()#.mean(dim=0)

In [60]:
down_features_1.shape

torch.Size([1421])

In [61]:
cosim = torch.nn.CosineSimilarity(dim=0)
sim_1 = cosim(down_features_1, down_features_2)
sim_2 = cosim(down_features_1, right_features_1)
sim_3 = cosim(right_features_1, right_features_2)
sim_4 = cosim(down_features_2, right_features_2)

sim_5 = cosim(stop_features_1, down_features_1)
sim_6 = cosim(stop_features_1, right_features_1)
sim_7 = cosim(stop_features_1, stop_features_2)
console.log(
    "Similarity\ndown_1 X down_2 : {}\ndown_1 X right_1 : {}\nright_1 X right_2 : {}\ndown_2 X right_2 : {}\nstop_1 X down_1 : {}\nstop_1 X right_1 : {}\nstop_1 X stop_2 : {}"
    .format(sim_1, sim_2, sim_3, sim_4, sim_5, sim_6, sim_7)
)

[09:12:19] Similarity                                                                              ]8;id=786922;file:///tmp/ipykernel_2244664/3292757301.py\3292757301.py]8;;\:]8;id=767455;file:///tmp/ipykernel_2244664/3292757301.py#10\10]8;;\
           down_1 X down_2 : 0.8353593349456787                                                                    
           down_1 X right_1 : 0.8317064046859741                                                                   
           right_1 X right_2 : -0.596428394317627                                                                  
           down_2 X right_2 : -0.5869486927986145                                                                  
           stop_1 X down_1 : 0.8446945548057556                                                                    
           stop_1 X right_1 : 0.8397865891456604                                                                   
           stop_1 X stop_2 : 0.7577064633369446                                                                    

In [58]:
cosim(stop_features_1, stop_features_2)

tensor(0.9542, device='cuda:0')

In [5]:
with torch.inference_mode():
    features, _ = model.extract_features(wav)
console.log(features[0].shape)

[08:29:55] torch.Size([1, 49, 768])                                                                 ]8;id=718360;file:///tmp/ipykernel_2244664/2654865961.py\2654865961.py]8;;\:]8;id=852616;file:///tmp/ipykernel_2244664/2654865961.py#3\3]8;;\

In [6]:
with torch.inference_mode():
    emission, _ = model(wav)
console.log(emission.shape, emission)

[08:29:57] torch.Size([1, 49, 29])                                                                  ]8;id=463475;file:///tmp/ipykernel_2244664/1127526247.py\1127526247.py]8;;\:]8;id=176755;file:///tmp/ipykernel_2244664/1127526247.py#3\3]8;;\
           tensor([[[ 11.5509,  -0.2031,  -0.9428,  ...,  -7.0372,  -6.1315,  -7.2063],                            
                    [ 11.5858,  -0.0873,  -0.9814,  ...,  -7.0390,  -6.2226,  -7.1677],                            
                    [ 11.4210,   0.4782,  -1.1381,  ...,  -6.7202,  -6.3945,  -6.6808],                            
                    ...,                                                                                           
                    [ -1.8378,   9.8166,   1.6956,  ...,  -5.9913,  -6.4387,  -5.1268],                            
                    [  3.7006,   9.6794,   1.0203,  ...,  -7.3436,  -8.5638,  -6.0118],                            
                    [ 11.0257,   7.6945,  -1.4180,  ...,  -8.6039, -10.0461,  -7.9190]]],                          
                  device='cuda:0')                                                                                 

In [7]:
decoder = GreedyCTCDecoder(labels=bundle.get_labels())
decoder = decoder.to(device)
transcript = decoder(emission[0])
console.log(transcript)

[08:29:58] DOWN|                                                                                    ]8;id=45463;file:///tmp/ipykernel_2244664/2457725976.py\2457725976.py]8;;\:]8;id=65946;file:///tmp/ipykernel_2244664/2457725976.py#4\4]8;;\

In [4]:
console.log(model)

[15:51:52] Wav2Vec2Model(                                                                            ]8;id=917896;file:///tmp/ipykernel_2604346/977660610.py\977660610.py]8;;\:]8;id=288307;file:///tmp/ipykernel_2604346/977660610.py#1\1]8;;\
             (feature_extractor): FeatureExtractor(                                                                
               (conv_layers): ModuleList(                                                                          
                 (0): ConvLayerBlock(                                                                              
                   (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)                                       
                   (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)                              
                 )                                                                                                 
                 (1-4): 4 x ConvLayerBlock(                                                                        
                   (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)                             
                 )                                                                                                 
                 (5-6): 2 x ConvLayerBlock(                                                                        
                   (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)                             
                 )                                                                                                 
               )                                                                                                   
             )                                                                                                     
             (encoder): Encoder(                                                                                   
               (feature_projection): FeatureProjection(                                                            
                 (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)                               
                 (projection): Linear(in_features=512, out_features=768, bias=True)                                
                 (dropout): Dropout(p=0.1, inplace=False)                                                          
               )                                                                                                   
               (transformer): Transformer(                                                                         
                 (pos_conv_embed): ConvolutionalPositionalEmbedding(                                               
                   (conv): Conv1d(768, 768, kernel_size=(128,), stride=(1,), padding=(64,),                        
           groups=16)                                                                                              
                 )                                                                                                 
                 (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)                               
                 (dropout): Dropout(p=0.1, inplace=False)                                                          
                 (layers): ModuleList(                                                                             
                   (0-11): 12 x EncoderLayer(                                                                      
                     (attention): SelfAttention(                                                                   
                       (dropout): Dropout(p=0.1, inplace=False)                                                    
                       (k_proj): Linear(in_features=768, out_features=768, bias=True)                              
                       (v_proj): Linear(

In [4]:
for param in model.feature_extractor.parameters():
    param.require_grad = False
for param in model.encoder.parameters():
    param.require_grad = False
for param in model.aux.parameters():
    param.require_grad = False
# model.aux = torch.nn.Sequential(
#     torch.nn.Flatten(),
#     torch.nn.Linear(in_features=37632, out_features=4096, device=device),
#     torch.nn.Linear(in_features=4096, out_features=8, bias=True, device=device)
#     
# )
# model.aux = torch.nn.Linear(in_features=768, out_features=8, bias=True, device=device)
aux = [model.aux]
# aux.extend([torch.nn.Linear(in_features=29, out_features=8, bias=True, device=device)])
aux.extend([torch.nn.Conv1d(in_channels=49, out_channels=8, kernel_size=29, stride=1, device=device)])
model.aux = torch.nn.Sequential(*aux)

In [5]:
console.log(model)

[15:58:31] Wav2Vec2Model(                                                                            ]8;id=601128;file:///tmp/ipykernel_2645657/977660610.py\977660610.py]8;;\:]8;id=477542;file:///tmp/ipykernel_2645657/977660610.py#1\1]8;;\
             (feature_extractor): FeatureExtractor(                                                                
               (conv_layers): ModuleList(                                                                          
                 (0): ConvLayerBlock(                                                                              
                   (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)                                       
                   (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)                              
                 )                                                                                                 
                 (1-4): 4 x ConvLayerBlock(                                                                        
                   (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)                             
                 )                                                                                                 
                 (5-6): 2 x ConvLayerBlock(                                                                        
                   (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)                             
                 )                                                                                                 
               )                                                                                                   
             )                                                                                                     
             (encoder): Encoder(                                                                                   
               (feature_projection): FeatureProjection(                                                            
                 (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)                               
                 (projection): Linear(in_features=512, out_features=768, bias=True)                                
                 (dropout): Dropout(p=0.1, inplace=False)                                                          
               )                                                                                                   
               (transformer): Transformer(                                                                         
                 (pos_conv_embed): ConvolutionalPositionalEmbedding(                                               
                   (conv): Conv1d(768, 768, kernel_size=(128,), stride=(1,), padding=(64,),                        
           groups=16)                                                                                              
                 )                                                                                                 
                 (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)                               
                 (dropout): Dropout(p=0.1, inplace=False)                                                          
                 (layers): ModuleList(                                                                             
                   (0-11): 12 x EncoderLayer(                                                                      
                     (attention): SelfAttention(                                                                   
                       (dropout): Dropout(p=0.1, inplace=False)                                                    
                       (k_proj): Linear(in_features=768, out_features=768, bias=True)                              
                       (v_proj): Linear(

In [21]:
model2 = torchaudio.models.wav2vec2_base(aux_num_out=8)
model2.aux = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(in_features=37632, out_features=4096, device=device),
    torch.nn.Linear(in_features=4096, out_features=8, bias=True, device=device)
    
)

In [22]:
model2.load_state_dict(model.state_dict())

<All keys matched successfully>

In [42]:
console.log(model2)

[18:09:23] Wav2Vec2Model(                                                                           ]8;id=892202;file:///tmp/ipykernel_1521672/1939789862.py\1939789862.py]8;;\:]8;id=411950;file:///tmp/ipykernel_1521672/1939789862.py#1\1]8;;\
             (feature_extractor): FeatureExtractor(                                                                
               (conv_layers): ModuleList(                                                                          
                 (0): ConvLayerBlock(                                                                              
                   (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)                                       
                   (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)                              
                 )                                                                                                 
                 (1-4): 4 x ConvLayerBlock(                                                                        
                   (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)                             
                 )                                                                                                 
                 (5-6): 2 x ConvLayerBlock(                                                                        
                   (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)                             
                 )                                                                                                 
               )                                                                                                   
             )                                                                                                     
             (encoder): Encoder(                                                                                   
               (feature_projection): FeatureProjection(                                                            
                 (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)                               
                 (projection): Linear(in_features=512, out_features=768, bias=True)                                
                 (dropout): Dropout(p=0.1, inplace=False)                                                          
               )                                                                                                   
               (transformer): Transformer(                                                                         
                 (pos_conv_embed): ConvolutionalPositionalEmbedding(                                               
                   (conv): Conv1d(768, 768, kernel_size=(128,), stride=(1,), padding=(64,),                        
           groups=16)                                                                                              
                 )                                                                                                 
                 (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)                               
                 (dropout): Dropout(p=0.1, inplace=False)                                                          
                 (layers): ModuleList(                                                                             
                   (0-11): 12 x EncoderLayer(                                                                      
                     (attention): SelfAttention(                                                                   
                       (dropout): Dropout(p=0.1, inplace=False)                                                    
                       (k_proj): Linear(in_features=768, out_features=768, bias=True)                              
                       (v_proj): Linea

In [6]:
from utils.audio_dataloader import MiniSpeechCommands
# from typing import Tuple, List, Dict
# from torch.utils.data import Dataset

data_dir = './datasets/speech-handsign_commands/speech'
TRAIN = 'train'
VAL = 'val'
TEST = 'test'

speech_datasets = {
    x: MiniSpeechCommands(os.path.join(data_dir, x))
    for x in [TRAIN, VAL, TEST]
}

dataloaders = {
    x: torch.utils.data.DataLoader(speech_datasets[x], batch_size=128, shuffle=True, num_workers=14)  #os.cpu_count() = 24
    for x in [TRAIN, VAL, TEST]
}

dataset_sizes = { x: len(speech_datasets[x]) for x in [TRAIN, VAL, TEST] }
class_names = speech_datasets[TRAIN].classes

for x in [TRAIN, VAL, TEST]:
    console.log("Loaded {} audios under {}".format(dataset_sizes[x], x))
console.log("Classes: ", class_names)

[15:58:41] Loaded 5760 audios under train                                                          ]8;id=269201;file:///tmp/ipykernel_2645657/2130267752.py\2130267752.py]8;;\:]8;id=667520;file:///tmp/ipykernel_2645657/2130267752.py#24\24]8;;\

           Loaded 640 audios under val                                                             ]8;id=661494;file:///tmp/ipykernel_2645657/2130267752.py\2130267752.py]8;;\:]8;id=894837;file:///tmp/ipykernel_2645657/2130267752.py#24\24]8;;\

           Loaded 1600 audios under test                                                           ]8;id=875739;file:///tmp/ipykernel_2645657/2130267752.py\2130267752.py]8;;\:]8;id=812203;file:///tmp/ipykernel_2645657/2130267752.py#24\24]8;;\

           Classes:                                                                                ]8;id=214359;file:///tmp/ipykernel_2645657/2130267752.py\2130267752.py]8;;\:]8;id=317499;file:///tmp/ipykernel_2645657/2130267752.py#25\25]8;;\
           ['down', 'go', 'left', 'no', 'right', 'stop', 'up', 'yes']                                              

In [12]:
inputs, _ = next(iter(dataloaders[TRAIN]))

In [13]:
inputs.shape

torch.Size([128, 16000])

In [14]:
outputs = model(inputs.to(device))

In [15]:
outputs[0].shape

torch.Size([128, 8])

In [16]:
outputs[0]

tensor([[ 0.0415,  0.0131,  0.0660,  ..., -0.0428, -0.0345,  0.1240],
        [ 0.1763,  0.0545, -0.0361,  ..., -0.0921,  0.1089,  0.1063],
        [-0.0022, -0.0502,  0.0143,  ...,  0.0030, -0.0025,  0.2192],
        ...,
        [-0.0282,  0.0153, -0.0244,  ..., -0.0377, -0.0734,  0.0102],
        [-0.0063,  0.0209, -0.0725,  ..., -0.0924,  0.1185,  0.1341],
        [ 0.0175,  0.0481,  0.0086,  ..., -0.0490,  0.0337,  0.0410]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [17]:
torch.softmax(outputs[0], dim=1).argmax(1).shape

torch.Size([128])

In [18]:
torch.softmax(outputs[0], dim=1).argmax(1)

tensor([7, 0, 7, 7, 7, 0, 7, 7, 7, 1, 7, 0, 7, 7, 7, 6, 7, 7, 7, 6, 7, 6, 7, 7,
        7, 7, 3, 7, 2, 7, 7, 7, 0, 7, 0, 1, 2, 0, 0, 7, 0, 7, 7, 7, 7, 7, 6, 7,
        7, 7, 7, 6, 7, 0, 0, 7, 2, 7, 6, 7, 1, 7, 6, 7, 6, 7, 7, 7, 7, 7, 7, 7,
        0, 0, 7, 6, 7, 0, 7, 7, 6, 0, 6, 3, 7, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 3, 7, 7, 7, 7, 7, 7, 0, 7, 7, 7, 7, 3, 7, 7, 7, 7, 7, 7, 0, 6, 7,
        7, 7, 7, 7, 7, 1, 7, 1], device='cuda:0')

In [7]:
def accuracy_fn(y_true: torch.Tensor, y_pred: torch.Tensor):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = correct/len(y_pred)
    return acc

def train(model, criterion, optimizer, scheduler, dataloader):
    model.train()
    losses, accs = 0, 0
    for i, batch in enumerate(dataloader):
        gc.collect()
        optimizer.zero_grad()
        wav, label = batch
        wav, label = wav.to(device), label.to(device).float()
        logits, _ = model(wav)
        pred = torch.softmax(logits.squeeze(2), dim=1).argmax(dim=1).float()
        # pred = torch.mean(logits, dim=1).softmax(dim=1).argmax(dim=1).float()
        # pred = torch.softmax(logits, dim=1).argmax(dim=1).float()
        loss = criterion(pred, label)
        loss.requires_grad = True
        acc = accuracy_fn(label, pred)
        losses += loss.item()
        accs += acc
        loss.backward()
        optimizer.step()
        scheduler.step()
    return {
        "train/loss": losses/dataset_sizes[TRAIN],
        "train/acc": accs/dataset_sizes[TRAIN]
    }

def validate(model, criterion, dataloader):
    model.eval()
    losses, accs = 0, 0
    for i, batch in enumerate(dataloader):
        gc.collect()
        wav, label = batch
        wav, label = wav.to(device), label.to(device).float()
        with torch.inference_mode():
            logits, _ = model(wav)
            pred = torch.softmax(logits.squeeze(2), dim=1).argmax(dim=1).float()
            # pred = torch.mean(logits, dim=1).softmax(dim=1).argmax(dim=1).float()
            # pred = torch.softmax(logits, dim=1).argmax(dim=1).float()
            loss = criterion(pred, label)
            acc = accuracy_fn(label, pred)
            losses += loss.item()
            accs += acc
    return {
        "val/loss": losses/dataset_sizes[VAL],
        "val/acc": accs/dataset_sizes[VAL]
    }

def evaluate(model, criterion, dataloader):
    model.eval()
    losses, accs = 0, 0
    for i, batch in enumerate(dataloader):
        gc.collect()
        wav, label = batch
        wav, label = wav.to(device), label.to(device).float()
        with torch.inference_mode():
            logits, _ = model(wav)
            pred = torch.softmax(logits.squeeze(2), dim=1).argmax(dim=1).float()
            # pred = torch.mean(logits, dim=1).softmax(dim=1).argmax(dim=1).float()
            # pred = torch.softmax(logits, dim=1).argmax(dim=1).float()
            loss = criterion(pred, label)
            acc = accuracy_fn(label, pred)
            losses += loss.item()
            accs += acc
    return {
        "eval/loss": losses/dataset_sizes[TEST],
        "eval/acc": accs/dataset_sizes[TEST]
    }

In [8]:
def run_optimization_loop():
    logger_fields = {
        "epoch": {},
        "tr/loss": {
            "goal": "lower_is_better",
            "format": "{:.6f}",
            "name": r"train/loss",
        },
        "va/loss": {
            "goal": "lower_is_better",
            "format": "{:.6f}",
            "name": r"val/loss",
        },
        "tr/acc": {
            "goal": "higher_is_better",
            "format": "{:.6f}",
            "name": r"train/acc",
        },
        "va/acc": {
            "goal": "higher_is_better",
            "format": "{:.6f}",
            "name": r"val/acc",
        },
        "duration": {"format": "{:.1f}", "name": "dur(s)"},
        ".*": True,  # Any other field must be logged at the end
    }
    printer = RichTablePrinter(key="epoch", fields=logger_fields)
    printer.hijack_tqdm()
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, betas=(.9, .98), eps= 1e-08)
    scheduler = lr_scheduler.StepLR(optimizer, step_size= 10, gamma= .1)
    
    gc.collect()
    start_t = time.time()
    for epoch in trange(100):
        if use_gpu:
            torch.cuda.empty_cache()
        t = time.time()
        train_metrics = train(model, criterion, optimizer, scheduler, dataloaders[TRAIN])
        printer.log(
            {
                "epoch": epoch,
                "tr/loss": train_metrics['train/loss'],
                "tr/acc": train_metrics['train/acc'],
            }
        )
        val_metrics = validate(model, criterion, dataloaders[VAL])
        printer.log(
            {
                "epoch": epoch,
                "va/loss": val_metrics['val/loss'],
                "va/acc": val_metrics['val/acc'],
                "duration": time.time() - t,
            }
        )
    printer.finalize()
    elapsed_time = time.time() - start_t
    console.log("Total Time Used : {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    start_e = time.time()
    eval_metrics = evaluate(model, criterion, dataloaders[TEST])
    elapsed_time = time.time() - start_e
    console.log("Average loss on Test set : {:.6f}".format(eval_metrics['eval/loss']))
    console.log("Average accuracy on Test set : {:.6f}".format(eval_metrics['eval/acc']))
    console.log("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    return model, scheduler, optimizer

In [ ]:
model, scheduler, optimizer = run_optimization_loop()

┏━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┓
┃ epoch ┃ train/loss ┃ val/loss  ┃ train/acc ┃ val/acc  ┃ dur(s) ┃
┡━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━┩
│ 0     │ 21.960777  │ 21.837253 │ 0.000997  │ 0.001025 │ 6.9    │
│ 1     │ 21.999228  │ 21.814946 │ 0.001080  │ 0.001025 │ 5.9    │
│ 2     │ 22.022519  │ 21.819518 │ 0.001059  │ 0.001025 │ 5.9    │
│ 3     │ 21.807635  │ 21.842120 │ 0.001093  │ 0.001025 │ 5.9    │
│ 4     │ 21.972453  │ 21.840737 │ 0.001038  │ 0.001025 │ 6.1    │
│ 5     │ 22.011145  │ 21.844699 │ 0.001005  │ 0.001025 │ 6.2    │
│ 6     │ 22.118203  │ 21.806759 │ 0.001013  │ 0.001025 │ 6.1    │
│ 7     │ 22.023744  │ 21.848636 │ 0.001046  │ 0.001025 │ 6.2    │
│ 8     │ 21.901803  │ 21.845760 │ 0.001039  │ 0.001025 │ 6.4    │
│ 9     │ 22.075481  │ 21.826322 │ 0.000972  │ 0.001025 │ 6.0    │
│ 10    │ 22.060517  │ 21.847754 │ 0.001047  │ 0.001025 │ 6.1    │
│ 11    │ 21.753146  │ 21.823861 │ 0.001065  │ 0.001025 │ 6.2    │
│ 12    │ 22.110190  │ 21.833380 │ 0.001053  │ 0.001025 │ 6.1    │
│ 13    │ 22.019134  │ 21.840486 │ 0.001005  │ 0.001025 │ 6.3    │
│ 14    │ 22.082112  │ 21.840646 │ 0.001029  │ 0.001025 │ 6.4    │
│ 15    │ 22.033630  │ 21.800688 │ 0.001051  │ 0.001025 │ 6.1    │
│ 16    │ 22.132235  │ 21.822047 │ 0.000991  │ 0.001025 │ 6.2    │
│ 17    │ 21.985966  │ 21.848256 │ 0.001082  │ 0.001025 │ 6.5    │
│ 18    │ 22.083195  │ 21.846689 │ 0.001039  │ 0.001025 │ 6.3    │
│ 19    │ 22.028708  │ 21.847018 │ 0.001013  │ 0.001025 │ 6.4    │
│ 20    │ 21.977361  │ 21.844926 │ 0.001025  │ 0.001025 │ 6.3    │
│ 21    │ 22.086658  │ 21.839880 │ 0.001020  │ 0.001025 │ 6.7    │
│ 22    │ 22.060078  │ 21.834441 │ 0.001004  │ 0.001025 │ 6.4    │
│ 23    │ 21.847501  │ 21.836887 │ 0.001072  │ 0.001025 │ 6.7    │
│ 24    │ 22.107481  │ 21.800050 │ 0.000974  │ 0.001025 │ 6.5    │
│ 25    │ 21.960583  │ 21.810136 │ 0.001057  │ 0.001025 │ 6.3    │
│ 26    │ 21.963554  │ 21.801702 │ 0.001057  │ 0.001025 │ 6.4    │
│ 27    │ 21.932799  │ 21.817315 │ 0.001009  │ 0.001025 │ 6.5    │
│ 28    │ 21.843423  │ 21.822698 │ 0.001047  │ 0.001025 │ 6.6    │
│ 29    │ 22.062059  │ 21.828049 │ 0.001027  │ 0.001025 │ 6.6    │
│ 30    │ 22.006302  │ 21.844309 │ 0.001017  │ 0.001025 │ 6.5    │
│ 31    │ 21.890476  │ 21.834391 │ 0.001090  │ 0.001025 │ 6.6    │
│ 32    │ 21.944794  │ 21.847700 │ 0.001039  │ 0.001025 │ 6.4    │
│ 33    │ 22.040176  │ 21.794580 │ 0.000978  │ 0.001025 │ 6.5    │
│ 34    │ 22.140384  │ 21.826681 │ 0.001043  │ 0.001025 │ 6.4    │
│ 35    │ 22.068311  │ 21.834175 │ 0.001040  │ 0.001025 │ 6.6    │
│ 36    │ 21.989203  │ 21.825790 │ 0.001074  │ 0.001025 │ 6.2    │
│ 37    │ 22.089188  │ 21.843402 │ 0.001010  │ 0.001025 │ 6.4    │
│ 38    │ 22.116336  │ 21.833327 │ 0.001021  │ 0.001025 │ 6.6    │
│ 39    │ 21.996094  │ 21.843421 │ 0.001065  │ 0.001025 │ 6.5    │
│ 40    │ 21.968361  │ 21.825582 │ 0.001024  │ 0.001025 │ 6.5    │
│ 41    │ 21.999423  │ 21.832989 │ 0.001054  │ 0.001025 │ 6.7    │
│ 42    │ 22.138747  │ 21.848404 │ 0.000989  │ 0.001025 │ 6.8    │
│ 43    │ 21.939850  │ 21.834260 │ 0.001074  │ 0.001025 │ 6.8    │
│ 44    │ 21.851898  │ 21.848591 │ 0.001085  │ 0.001025 │ 6.7    │
│ 45    │ 21.902458  │ 21.824277 │ 0.001065  │ 0.001025 │ 6.9    │
│ 46    │ 22.008917  │ 21.841195 │ 0.000996  │ 0.001025 │ 6.7    │
│ 47    │ 22.046900  │ 21.844176 │ 0.001062  │ 0.001025 │ 6.3    │
│ 48    │ 21.944022  │ 21.840653 │ 0.001048  │ 0.001025 │ 6.6    │
│ 49    │ 22.058904  │ 21.846817 │ 0.001038  │ 0.001025 │ 6.8    │
│ 50    │ 21.952494  │ 21.805006 │ 0.001059  │ 0.001025 │ 6.6    │
│ 51    │ 22.114142  │ 21.845915 │ 0.001036  │ 0.001025 │ 6.9    │
│ 52    │ 21.939853  │ 21.822189 │ 0.001050  │ 0.001025 │ 6.4    │
│ 53    │ 21.977340  │ 21.818811 │ 0.001076  │ 0.001025 │ 6.4    │
│ 54    │ 21.958647  │ 21.843050 │ 0.001016  │ 0.001025 │ 6.5    │
│ 55    │ 21.973229  │ 21.827525 │ 0.001039  │ 0.001025 │ 6.5    │
│ 56    │ 22.143959  │ 21.825645 │ 0.001000  │ 

In [15]:
out, _ = model(wav)
pred = torch.softmax(out, dim=1).argmax(dim=1).float()
console.log(pred)

           tensor([5.], device='cuda:0')                                                             ]8;id=115518;file:///tmp/ipykernel_1760775/873647403.py\873647403.py]8;;\:]8;id=985729;file:///tmp/ipykernel_1760775/873647403.py#3\3]8;;\